# [Scrapy Beginners Series Part 1: How To Build Your First Production Scraper](https://scrapeops.io/python-scrapy-playbook/scrapy-beginners-guide/)

# Scrapy 初心者向けシリーズ パート 1: 初めてのプロダクション スクレーパーを構築する方法

- Python Scrapy 5部構成の初心者向けシリーズ
- Part 1: Basic Scrapy Spider
  - Scrapyの基本を確認し、最初のScrapyスパイダーを構築します。
- Part 2：汚れたデータのクリーニングとエッジケースの処理
  - Webデータは厄介で構造化されておらず、多くのエッジケースを持つことができます。
  - Items、Itemloaders、Item Pipelineを使用して、これらのエッジケースに対してスパイダーを堅牢にします。
- Part 3：データの保存
  - データベース、CSVファイル、JSONフォーマット、S3バケットからスクレイピングしたデータを保存する方法はさまざまです。
  - データを保存するさまざまな方法を検討し、その長所、短所、どのような場面で使うかについて説明します。
- Part4：ユーザーエージェントとプロキシ
  - ユーザーエージェントとIPを管理し、ブロックされないようにすることで、スパイダーを本番環境に対応させます。
- Part 5: デプロイメント、スケジューリング、ジョブの実行
  - サーバー上にスパイダーをデプロイし、[ScrapeOps](https://scrapeops.io/)を介してジョブの監視とスケジューリングを行う。

- CSSセレクタとXPath表現の解析
- データ形式（CSV、JSON、XML）およびストレージ（FTP、S3、ローカルファイルシステム）
- 堅牢なエンコーディングサポート
- 同時実行管理
- 自動再試行
- クッキーとセッションの処理
- クロールスパイダーと内蔵のページネーションサポート

## Beginners Scrapy Tutorial

### Step 1 - Setup your Python Environment

#### Test Scrapy Is Installed

```bash
(venv) c01 (😁 :main *) :$ scrapy

Scrapy 2.9.0 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command
```

### Step 2 - Setup Our Scrapy Project

```bash
# scrapyプロジェクトの作成
(venv) c01 (😁 :main *) :$ scrapy startproject chocolatescraper

New Scrapy project 'chocolatescraper', using template directory '/Users/takeru/@LEARNING/Python/python-scrapy-playbook-scrapyops/venv/lib/python3.10/site-packages/scrapy/templates/project', created in:
    /Users/takeru/@LEARNING/Python/python-scrapy-playbook-scrapyops/c01/chocolatescraper

You can start your first spider with:
    cd chocolatescraper
    scrapy genspider example example.com
```

```bash
# chocolatescraperディレクトリに移動
cd chocolatescraper
```

```bash
# chocolatescraperディレクトリのtree
(venv) chocolatescraper (😁 :main *) :$ tree

.
├── chocolatescraper
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

3 directories, 7 files
```

- `settings.py`
  - パイプラインやミドルウェアの有効化など、プロジェクトのすべての設定が含まれる場所です。
  - 遅延、同時実行、その他多くのことを変更することができます。
- `items.py`
  - 抽出されたデータのモデルです。
  - Scrapy Itemクラスを継承し、スクレイピングされたデータを含むカスタムモデル（ProductItemのような）を定義することができます
- `pipelines.py`
  - スパイダーが生成したアイテムが渡される場所で、主にテキストのクリーニングやファイル出力やデータベース（CSV、JSON SQLなど）への接続に使用します。
- `middlewares.py`
  - リクエストが行われ、scrapy がレスポンスを処理する方法を変更したいときに便利です。
- `scrapy.cfg`
  - いくつかのデプロイ設定などを変更するための設定ファイルです。

### Step 3- Creating Our Spider

- Spider
  - `start_url` のリストを受け取り、それぞれを `parse` メソッドでスクレイピングする。
- CrawlSpider 
  - 見つけたリンクをたどって、ウェブサイト全体をクロールするように設計されている。
- SitemapSpider
  - サイトマップから URL を抽出するように設計されてる。

```bash
# 新しい汎用スパイダーを作成するには、`genspider`コマンドを実行するだけです：
# scrapy genspider <name_of_spider> <website>
(venv) chocolatescraper (😁 :main *) :$ scrapy genspider chocolatespider chocolate.co.uk

Created spider 'chocolatespider' using template 'basic' in module:
  chocolatescraper.spiders.chocolatespider
```

これで `spiders` フォルダに新しいスパイダーが追加され、次のようになります：

```python
# chocolatespider.py

import scrapy

class ChocolatespiderSpider(scrapy.Spider):
    name = 'chocolatespider'
    allowed_domains = ['chocolate.co.uk']
    start_urls = ['http://chocolate.co.uk/']

    def parse(self, response):
        pass
```

ここでは、`genspider`コマンドが、`Spider`クラスという形で、私たちが使用するためのテンプレートスパイダーを作成したことを確認します。

このスパイダー・クラスには以下のものがあります：

- name
  - スパイダーに名前をつけるクラス属性です。
  - あとで `scrapy crawl <spider_name>` としてスパイダーを実行するときに、この名前を使用します。
- allowed_domains
  - クラス属性で、Scrapyに `chocolate.co.uk` ドメインのページのみをスクレイピングするように指示します。
  - スパイダーが暴走して多くのウェブサイトをスクレイピングするのを防ぐためです。
  - これはオプションです。
- start_urls
  - Scrapyにスクレイピングすべき最初のURLを指示するクラス属性です。
  - これは少しづつ変更していきます。
- parse
  - ターゲットウェブサイトからレスポンスを受信した後、`parse`関数が呼び出されます。

このSpiderを使い始めるには、2つのことをする必要があります：

- start_urls`をスクレイピングしたいURL <https://www.chocolate.co.uk/collections/all>に変更する。
- パースコードを `parse` 関数に挿入する。

### Step 4 - Update Start Urls

```python
# chocolatespider.py

import scrapy

class ChocolatespiderSpider(scrapy.Spider):
    name = 'chocolatespider'
    allowed_domains = ['chocolate.co.uk']
    start_urls = ['https://chocolate.co.uk/collections/all']

    def parse(self, response):
        pass
```

次に、ページから必要なデータを解析するためのCSSセレクタを作成する必要があります。

これを行うには、Scrapy Shellを使用します。

### Step 5 - Scrapy Shell: Finding Our CSS Selectors

HTMLページからデータを抽出するには、[XPath](https://www.w3schools.com/xml/xpath_intro.asp)または[CSSセレクタ](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Selectors)を使用して、Scrapyにページのどこにデータがあるのかを伝える必要があります。XPathとCSSセレクタは、ScrapyがDOMツリーをナビゲートし、必要なデータの場所を見つけるための小さなマップのようなものです。このガイドでは、ページからデータを解析するためにCSSセレクタを使用するつもりです。そして、これらのCSSセレクタを作成するために、私たちは[Scrapy Shell](https://docs.scrapy.org/en/latest/topics/shell.html)を使用する予定です。

Scrapyの素晴らしい機能の1つは、あなたがすぐにあなたのXPathとCSSセレクタをテストし、デバッグすることができ、組み込みのシェルが付属していることです。XPathやCSSのセレクタが正しいかどうかを確認するためにスクレイパーを実行する代わりに、ターミナルに直接入力して結果を確認することができます。

```bash
# Scrapyシェルを開くには、次のコマンドを使用します：
(venv) chocolatescraper (😁 :main *) :$ scrapy shell

2023-05-29 20:06:24 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-05-29 20:06:24 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-05-29 20:06:24 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-05-29 20:06:24 [asyncio] DEBUG: Using selector: KqueueSelector
2023-05-29 20:06:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-05-29 20:06:24 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-05-29 20:06:24 [scrapy.extensions.telnet] INFO: Telnet Password: f3038c0cc138d466
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-05-29 20:06:24 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-05-29 20:06:24 [asyncio] DEBUG: Using selector: KqueueSelector
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x10dcf76a0>
[s]   item       {}
[s]   settings   <scrapy.settings.Settings object at 0x10dcf6ce0>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
2023-05-29 20:06:25 [asyncio] DEBUG: Using selector: KqueueSelector
In [1]: 
```

```bash
# scrapy shellを抜ける
In [1]: quit
```

#### Fetch The Page

CSSセレクタを作成するために、次のページでテストします：

<https://www.chocolate.co.uk/collections/all>